In [1]:
import pymongo
import datetime
import collections

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.stats
from collections import Counter
from sklearn import cluster

# Importing and cleaning our datasets

In [2]:
referendum_df = pd.read_csv("data/EU-referendum-result-data.csv")
referendum_df.head()

,id,Region_Code,Region,Area_Code,Area,Electorate,ExpectedBallots,VerifiedBallotPapers,Pct_Turnout,Votes_Cast,...,Remain,Leave,Rejected_Ballots,No_official_mark,Voting_for_both_answers,Writing_or_mark,Unmarked_or_void,Pct_Remain,Pct_Leave,Pct_Rejected
0,108,E12000006,East,E06000031,Peterborough,120892,87474,87469,72.35,87469,...,34176,53216,77,0,32,7,38,39.11,60.89,0.09
1,109,E12000006,East,E06000032,Luton,127612,84633,84636,66.31,84616,...,36708,47773,135,0,85,0,50,43.45,56.55,0.16
2,112,E12000006,East,E06000033,Southend-on-Sea,128856,93948,93939,72.90,93939,...,39348,54522,69,0,21,0,48,41.92,58.08,0.07
3,113,E12000006,East,E06000034,Thurrock,109897,79969,79954,72.75,79950,...,22151,57765,34,0,8,3,23,27.72,72.28,0.04
4,110,E12000006,East,E06000055,Bedford,119530,86136,86136,72.06,86135,...,41497,44569,69,0,26,1,42,48.22,51.78,0.08


In [3]:
immigration_df = pd.read_csv("data/ukmye2015/MYEB3_summary_components_of_change_series_UK_(0215).csv", thousands=',')
immigration_df.T.head()

,0,1,2,3,4,5,6,7,8,9,...,381,382,383,384,385,386,387,388,389,390
lad2014_code,E06000001,E06000002,E06000003,E06000004,E06000005,E06000006,E06000007,E06000008,E06000009,E06000010,...,W06000014,W06000015,W06000016,W06000018,W06000019,W06000020,W06000021,W06000022,W06000023,W06000024
lad2014_name,Hartlepool,Middlesbrough,Redcar and Cleveland,Stockton-on-Tees,Darlington,Halton,Warrington,Blackburn with Darwen,Blackpool,"Kingston upon Hull, City of",...,Vale of Glamorgan,Cardiff,Rhondda Cynon Taf,Caerphilly,Blaenau Gwent,Torfaen,Monmouthshire,Newport,Powys,Merthyr Tydfil
country,E,E,E,E,E,E,E,E,E,E,...,W,W,W,W,W,W,W,W,W,W
population_2001,90152,141233,139159,183795,97894,118559,191202,138453,142270,249913,...,119277,310088,231910,169546,70000,90912,84984,137642,126398,56207
births_2002,1017,1663,1336,1938,1102,1389,2149,2126,1417,2814,...,1169,3537,2464,1945,655,971,788,1656,1169,600


In [4]:
immigration_df.groupby("country").count()

,lad2014_code,lad2014_name,population_2001,births_2002,deaths_2002,natchange_2002,internal_in_2002,internal_out_2002,internal_net_2002,international_in_2002,...,deaths_2015,natchange_2015,internal_in_2054,internal_out_2015,internal_net_2015,international_in_2015,international_out_2015,international_net_2015,other_change_2015,population_2015
country,,,,,,,,,,,,,,,,,,,,,
E,326,326,326,326,326,326,326,326,326,326,...,326,326,326,326,326,326,326,326,326,326
N,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
S,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,32,32,32,32,32
W,22,22,22,22,22,22,22,22,22,22,...,22,22,22,22,22,22,22,22,22,22


In [5]:
# Let's have a look at the data types in our datasets:
referendum_df.dtypes

# It seems that all the values within the vote columns have been imported as int64 or float64, which might come in handy later 
# on when performing calculations with them.

id                           int64
Region_Code                 object
Region                      object
Area_Code                   object
Area                        object
Electorate                   int64
ExpectedBallots              int64
VerifiedBallotPapers         int64
Pct_Turnout                float64
Votes_Cast                   int64
Valid_Votes                  int64
Remain                       int64
Leave                        int64
Rejected_Ballots             int64
No_official_mark             int64
Voting_for_both_answers      int64
Writing_or_mark              int64
Unmarked_or_void             int64
Pct_Remain                 float64
Pct_Leave                  float64
Pct_Rejected               float64
dtype: object

In [6]:
referendum_df

,id,Region_Code,Region,Area_Code,Area,Electorate,ExpectedBallots,VerifiedBallotPapers,Pct_Turnout,Votes_Cast,...,Remain,Leave,Rejected_Ballots,No_official_mark,Voting_for_both_answers,Writing_or_mark,Unmarked_or_void,Pct_Remain,Pct_Leave,Pct_Rejected
0,108,E12000006,East,E06000031,Peterborough,120892,87474,87469,72.35,87469,...,34176,53216,77,0,32,7,38,39.11,60.89,0.09
1,109,E12000006,East,E06000032,Luton,127612,84633,84636,66.31,84616,...,36708,47773,135,0,85,0,50,43.45,56.55,0.16
2,112,E12000006,East,E06000033,Southend-on-Sea,128856,93948,93939,72.90,93939,...,39348,54522,69,0,21,0,48,41.92,58.08,0.07
3,113,E12000006,East,E06000034,Thurrock,109897,79969,79954,72.75,79950,...,22151,57765,34,0,8,3,23,27.72,72.28,0.04
4,110,E12000006,East,E06000055,Bedford,119530,86136,86136,72.06,86135,...,41497,44569,69,0,26,1,42,48.22,51.78,0.08
5,111,E12000006,East,E06000056,Central Bedfordshire,204004,158904,158896,77.89,158894,...,69670,89134,90,0,34,1,55,43.87,56.13,0.06
6,184,E12000006,East,E07000008,Cambridge,80108,57871,57860,72.22,57852,...,42682,15117,53,0,13,0,40,73.85,26.15,0.09
7,185,E12000006,East,E07000009,East Cambridgeshire,62435,48129,48120,77.08,48124,...,23599,24487,38,0,18,0,20,49.08,50.92,0.08
8,186,E12000006,East,E07000010,Fenland,71447,52653,52649,73.69,52649,...,15055,37571,23,0,10,1,12,28.61,71.39,0.04
9,187,E12000006,East,E07000011,Huntingdonshire,128486,99996,99990,77.82,99990,...,45729,54198,63,0,31,0,32,45.76,54.24,0.06


The referendum dataset was previously explored in the 'c8666430_TMA02_Question2b' notebook attached to the TMA02 by myself, so I do not think there is any need for a further explanation. Let's focus on the new dataset to explore, the immigration_df DataFrame:

In [7]:
immigration_df.dtypes


# However, when looking at columns within the immigration_df that should have been imported as int64, we observe that some
# of them (the ones corresponding to 2015) have been parsed as objects. Let's investigate why this happened

lad2014_code              object
lad2014_name              object
country                   object
population_2001            int64
births_2002                int64
deaths_2002                int64
natchange_2002             int64
internal_in_2002           int64
internal_out_2002          int64
internal_net_2002          int64
international_in_2002      int64
international_out_2002     int64
international_net_2002     int64
other_change_2002          int64
population_2002            int64
births_2003                int64
deaths_2003                int64
natchange_2003             int64
internal_in_2003           int64
internal_out_2003          int64
internal_net_2003          int64
international_in_2003      int64
international_out_2003     int64
international_net_2003     int64
other_change_2003          int64
population_2003            int64
births_2004                int64
deaths_2004                int64
natchange_2004             int64
internal_in_2004           int64
          

In [8]:
#Let's have a look at the values of one of the 'object' columns:

immigration_df["international_in_2015"].unique()

array(['248', '1250', '202', '994', '347', '249', '1175', '778', '565',
       '2718', '975', '485', '838', '2889', '2369', '7611', '148', '7449',
       '1996', '810', '1764', '2610', '7016', '788', '1750', '2129', '491',
       '3819', '846', '1801', '3129', '4192', '811', '1001', '1570', '927',
       '800', '3565', '2156', '1281', '1153', '2238', '6688', '2892',
       '5348', '421', '2115', '1607', '1436', '996', '2282', '22', '2076',
       '1511', '1841', '530', '1696', '406', '420', '1099', '4796', '769',
       '1199', '995', '201', '111', '376', '100', '185', '573', '200',
       '307', '196', '159', '188', '145', '180', '419', '2553', '286',
       '334', '336', '183', '181', '138', '209', '310', '144', '374',
       '218', '477', '357', '298', '552', '581', '458', '305', '140',
       '883', '2174', '585', '752', '142', '116', '319', '335', '1081',
       '472', '236', '910', '328', '423', '986', '438', '325', '393',
       '259', '648', '760', '521', '875', '457', '765', '

In [9]:
# We observe a strange value close to the end of the list of unique values. There is a cell that contains something like:'    :'.
# This might represent missing data or an error when the data was entered. We should now decide how to deal with this. 
# For now let's have a look at the row containing such error.

columns2015 = [column for column in immigration_df.columns if "2015" in column]
immigration_df.loc[immigration_df["international_in_2015"] == '                  :'][columns2015]

# Ok. It seems to happen that the data for Northern Ireland for 2015 is somehow missing. 
# We will now changed the '                  :' values.

,births_2015,deaths_2015,natchange_2015,internal_out_2015,internal_net_2015,international_in_2015,international_out_2015,international_net_2015,other_change_2015,population_2015
326,:,:,:,:,:,:,:,:,:,:
327,:,:,:,:,:,:,:,:,:,:
328,:,:,:,:,:,:,:,:,:,:
329,:,:,:,:,:,:,:,:,:,:
330,:,:,:,:,:,:,:,:,:,:
331,:,:,:,:,:,:,:,:,:,:
332,:,:,:,:,:,:,:,:,:,:
333,:,:,:,:,:,:,:,:,:,:
334,:,:,:,:,:,:,:,:,:,:
335,:,:,:,:,:,:,:,:,:,:


In [10]:
# In order to decide what to do with this data let's first have a look at the impact of the missing data (Let's check how many 
# columns are affected)

def count_errors(row):
    return sum(row == '                  :')

print(immigration_df.apply(count_errors))
print(immigration_df.apply(count_errors).sum())

# So the error value '                  :' seems to affect only to the 2015 columns for the North Ireland rows. The problem is
# that it affects every single Northern Ireland row.

lad2014_code               0
lad2014_name               0
country                    0
population_2001            0
births_2002                0
deaths_2002                0
natchange_2002             0
internal_in_2002           0
internal_out_2002          0
internal_net_2002          0
international_in_2002      0
international_out_2002     0
international_net_2002     0
other_change_2002          0
population_2002            0
births_2003                0
deaths_2003                0
natchange_2003             0
internal_in_2003           0
internal_out_2003          0
internal_net_2003          0
international_in_2003      0
international_out_2003     0
international_net_2003     0
other_change_2003          0
population_2003            0
births_2004                0
deaths_2004                0
natchange_2004             0
internal_in_2004           0
                          ..
internal_in_2013           0
internal_out_2013          0
internal_net_2013          0
international_

In [11]:
# for now we will just declare '                  :' as Null value. Later on we will decide what to do about it.
immigration_df.replace(to_replace='                  :', value=np.nan, inplace=True)

# let's now check if the values have indeed changed:
immigration_df.loc[immigration_df["international_in_2015"].isnull()][columns2015]

,births_2015,deaths_2015,natchange_2015,internal_out_2015,internal_net_2015,international_in_2015,international_out_2015,international_net_2015,other_change_2015,population_2015
326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Here I found another error: some numbers were imported with commas to indicate thousands. To fix it I went back to the original import and added the "thousands=','" parameter to the read_csv() method

In [12]:
#Let's now have a look at its columns'
immigration_df.columns

Index(['lad2014_code', 'lad2014_name', 'country', 'population_2001',
       'births_2002', 'deaths_2002', 'natchange_2002', 'internal_in_2002',
       'internal_out_2002', 'internal_net_2002',
       ...
       'deaths_2015', 'natchange_2015', 'internal_in_2054',
       'internal_out_2015', 'internal_net_2015', 'international_in_2015',
       'international_out_2015', 'international_net_2015', 'other_change_2015',
       'population_2015'],
      dtype='object', length=158)

At first sight we observe three clearly different columns - "lad2014_code", "lad2014_name" and "country" - which only appear
once. If we have a look at the "MYEB_information_note" document we learn that "lad2014_code" corresponds to the ONS Geographic code for local authorities as defined at mid-2014; "lad2014_name" is the local district or unitary authority name; and country stands for "E": England, "W": Wales, "S": Scotland and "N": Northern Ireland.

The rest of the columns represent population, birht estimates, death estimates, in and out national and international migratory moves and their net values (difference between in and out) for each year from 2001 to 2015.

From the list of columns above we observe what seems to be a typo within one of the 2014 columns: There is a column named "internal_in_2054" although it obviously should read "internal_in_2014". Let's start by fixing this:

In [13]:
immigration_df.columns = [w.replace("internal_in_2054", "internal_in_2015") for w in immigration_df.columns]
# Let's see if the column name has been changed:
immigration_df.columns

Index(['lad2014_code', 'lad2014_name', 'country', 'population_2001',
       'births_2002', 'deaths_2002', 'natchange_2002', 'internal_in_2002',
       'internal_out_2002', 'internal_net_2002',
       ...
       'deaths_2015', 'natchange_2015', 'internal_in_2015',
       'internal_out_2015', 'internal_net_2015', 'international_in_2015',
       'international_out_2015', 'international_net_2015', 'other_change_2015',
       'population_2015'],
      dtype='object', length=158)

We can now see that the column reads "internal_in_2014".

Now, let's rename some columns so the names are more friendly to work with:

In [14]:
immigration_df = immigration_df.rename(columns = {'lad2014_code':'Area_Code', 'lad2014_name':'Area'})
immigration_df

,Area_Code,Area,country,population_2001,births_2002,deaths_2002,natchange_2002,internal_in_2002,internal_out_2002,internal_net_2002,...,deaths_2015,natchange_2015,internal_in_2015,internal_out_2015,internal_net_2015,international_in_2015,international_out_2015,international_net_2015,other_change_2015,population_2015
0,E06000001,Hartlepool,E,90152,1017,1042,-25,2140,2301,-161,...,1102,-81,2137,2281,-144,248,135,113,15,92493
1,E06000002,Middlesbrough,E,141233,1663,1396,267,5109,6737,-1628,...,1488,465,6184,7071,-887,1250,450,800,12,139509
2,E06000003,Redcar and Cleveland,E,139159,1336,1523,-187,3951,4259,-308,...,1549,-80,4214,4019,195,202,109,93,25,135275
3,E06000004,Stockton-on-Tees,E,183795,1938,1719,219,6269,5531,738,...,1874,487,6442,6659,-217,994,557,437,-23,194803
4,E06000005,Darlington,E,97894,1102,1173,-71,3815,3521,294,...,1135,99,3743,3779,-36,347,407,-60,19,105389
5,E06000006,Halton,E,118559,1389,1230,159,3253,3678,-425,...,1240,302,3470,3646,-176,249,217,32,16,126528
6,E06000007,Warrington,E,191202,2149,1836,313,6517,6412,105,...,2000,350,7127,6650,477,1175,722,453,-13,207695
7,E06000008,Blackburn with Darwen,E,138453,2126,1383,743,4792,5665,-873,...,1277,882,4754,5631,-877,778,673,105,-7,146846
8,E06000009,Blackpool,E,142270,1417,2135,-718,9023,8333,690,...,1954,-176,7480,8241,-761,565,561,4,10,139578
9,E06000010,"Kingston upon Hull, City of",E,249913,2814,2584,230,9009,10690,-1681,...,2628,873,8715,9815,-1100,2718,1467,1251,261,258995


# Aggregating data over the years

### Calculating incoming immigration over the years

In [15]:
# Here I am going to try to calculate the international in total for the international migrations for the whole dataframe. 
# Since some data from 2015 presents missing data we will ignore it for now. We will limit our study to the years 2001 - 2014. 
# We will first create a variable to hold the columns that we are intersted in, which are the international_in_year:
international_in_columns = [column for column in immigration_df if "international_in" in column]
international_in_columns

['international_in_2002',
 'international_in_2003',
 'international_in_2004',
 'international_in_2005',
 'international_in_2006',
 'international_in_2007',
 'international_in_2008',
 'international_in_2009',
 'international_in_2010',
 'international_in_2011',
 'international_in_2012',
 'international_in_2013',
 'international_in_2014',
 'international_in_2015']

In [16]:
# We won't use the last item of the list: "international_in_2015" for the reasons stated above. Let's eliminate it:
international_in_columns.remove("international_in_2015")
international_in_columns

['international_in_2002',
 'international_in_2003',
 'international_in_2004',
 'international_in_2005',
 'international_in_2006',
 'international_in_2007',
 'international_in_2008',
 'international_in_2009',
 'international_in_2010',
 'international_in_2011',
 'international_in_2012',
 'international_in_2013',
 'international_in_2014']

In [17]:
# Let's now create a function that adds all the international_in_year values and assigns their results to a new column
# called international_in_total
def sum_international_in(row):
    return row[international_in_columns].sum()
immigration_df["international_in_total"] = immigration_df.apply(sum_international_in, axis=1)
immigration_df["international_in_total"].head()
# We can see now our new column holding data for the international_net total added from 2002 to 2014

0     2524
1    15883
2     3501
3     9454
4     4671
Name: international_in_total, dtype: int64

In [18]:
immigration_df[["Area","international_in_total"]].head()

,Area,international_in_total
0,Hartlepool,2524
1,Middlesbrough,15883
2,Redcar and Cleveland,3501
3,Stockton-on-Tees,9454
4,Darlington,4671


### Calculating net immigration over the years

In [19]:
international_net_columns = [column for column in immigration_df if "international_net" in column]
international_net_columns

['international_net_2002',
 'international_net_2003',
 'international_net_2004',
 'international_net_2005',
 'international_net_2006',
 'international_net_2007',
 'international_net_2008',
 'international_net_2009',
 'international_net_2010',
 'international_net_2011',
 'international_net_2012',
 'international_net_2013',
 'international_net_2014',
 'international_net_2015']

In [20]:
international_net_columns.remove("international_net_2015")
international_net_columns

['international_net_2002',
 'international_net_2003',
 'international_net_2004',
 'international_net_2005',
 'international_net_2006',
 'international_net_2007',
 'international_net_2008',
 'international_net_2009',
 'international_net_2010',
 'international_net_2011',
 'international_net_2012',
 'international_net_2013',
 'international_net_2014']

In [21]:
def sum_international_net(row):
    return row[international_net_columns].sum()
immigration_df["international_net_total"] = immigration_df.apply(sum_international_net, axis=1)
immigration_df["international_net_total"]

0        546
1       8376
2         40
3       1953
4        304
5       -615
6       3905
7       3822
8        444
9      21669
10      5763
11      2874
12      6524
13     15761
14     18013
15     38992
16       359
17     47519
18      8526
19      3801
20     10226
21     10006
22     36890
23      3695
24      9856
25      8625
26      -378
27      6955
28       714
29     10335
       ...  
361     1365
362     -485
363      -53
364    -1371
365    -2914
366     3519
367     -609
368     3239
369      341
370     3891
371     -490
372      262
373      896
374     5390
375     1644
376      767
377     2926
378     9558
379       76
380      654
381      -64
382    19765
383     2504
384     -598
385      248
386      -89
387     -659
388     2058
389      454
390     1145
Name: international_net_total, dtype: int64

In [22]:
immigration_df[["Area","international_net_total"]].head()

,Area,international_net_total
0,Hartlepool,546
1,Middlesbrough,8376
2,Redcar and Cleveland,40
3,Stockton-on-Tees,1953
4,Darlington,304


### Calculating population change over the years

In [23]:
def calculate_population_change(row):
    return (row["population_2014"] - row["population_2001"])
immigration_df["population_change"] = immigration_df.apply(calculate_population_change, axis=1)
immigration_df["population_change"]

0       2438
1      -2114
2      -4117
3      10324
4       7473
5       7795
6      15226
7       8290
8      -1769
9       7797
10     22261
11      1853
12     16283
13     23148
14     21737
15     54896
16      3424
17     45329
18     12275
19     10867
20     10605
21     12863
22     52425
23     19314
24     25571
25     20592
26      3019
27     27830
28     11741
29     35670
       ...  
361     1380
362     7090
363     1260
364      370
365    12960
366     5320
367    -3610
368    18170
369     2363
370     5429
371     6613
372     1721
373     5175
374     8174
375        8
376    10608
377    11246
378    17834
379     6110
380    12479
381     8408
382    44206
383     4978
384    10395
385     -326
386      697
387     7352
388     9199
389     6277
390     2858
Name: population_change, dtype: int64

# Different rows
Let's here have a look at the datasets looking for rows that might appear **only** in one of them

In [24]:
# First, let's extract the set of values for the columns "Area_Code" for both datasets.
referendum_codes = referendum_df["Area_Code"].unique()
set_referendum_codes = set(referendum_codes.flatten())
for code in set_referendum_codes:
    print(code)
print(len(set_referendum_codes))

E07000005
E07000150
E07000238
E07000148
E07000131
E07000153
E07000091
E07000010
E07000089
E07000073
E08000014
E08000021
E06000037
E07000109
E07000211
E07000241
E06000008
E06000050
E07000093
E07000204
S12000038
E06000009
E07000206
E08000006
E07000167
W06000015
S12000023
E08000002
S12000046
E08000034
E07000225
E08000029
E07000066
E07000031
E07000044
E08000012
W06000022
E07000195
E09000027
E07000120
E09000019
E08000033
E07000045
E07000174
E07000152
E07000028
W06000013
S12000041
E07000124
E06000011
E08000031
E06000017
E07000117
E07000111
E07000228
W06000014
E07000132
E07000194
W06000011
S12000028
E06000025
E09000007
E06000006
S12000018
E07000061
E07000069
E09000014
E06000052
E07000128
E07000201
E08000003
E07000236
E07000137
E07000027
E07000129
E07000086
E07000041
E08000030
E07000243
E06000040
E08000005
E07000063
E07000006
E06000026
E06000031
E07000193
E07000168
E06000007
E08000007
E07000039
E06000015
E07000077
E07000209
E06000030
E06000023
E07000219
E07000178
E06000027
E09000021
E06000021


In [25]:
immigration_codes = immigration_df["Area_Code"].unique()
set_immigration_codes = set(immigration_codes.flatten())
for code in set_immigration_codes:
    print(code)
print(len(set_immigration_codes))

N09000001
E07000005
E07000150
E07000238
E07000148
E07000131
E07000153
E07000091
E07000010
E07000089
E07000073
E08000014
E08000021
E06000037
E07000109
E07000211
E07000241
E06000008
E06000050
E07000093
E07000204
S12000038
E06000009
E07000206
E08000006
E07000167
W06000015
S12000023
E08000002
S12000046
E08000034
E07000225
E08000029
E07000066
E07000031
E07000044
E08000012
W06000022
E07000195
E09000027
E07000120
E08000033
E09000019
E07000045
E07000174
E07000152
E07000028
W06000013
S12000041
E07000124
E06000011
E08000031
E06000017
E07000117
E07000111
E07000228
W06000014
E07000132
N09000003
E07000194
W06000011
S12000028
E06000025
E09000007
E06000006
E07000061
S12000018
E07000069
E06000052
E09000014
E07000128
E07000201
E08000003
E07000236
N09000010
E07000137
E07000027
E07000129
E07000086
E07000041
E08000030
N09000004
E07000243
E06000040
E08000005
E07000063
E07000006
E06000026
E06000031
E07000193
N09000002
E07000168
E06000007
E08000007
E07000039
E06000015
E07000077
E07000209
E06000030
E06000023


In [26]:
# Now let's extract the difference of both sets (values that appear only in one of the sets)
set_codes_missing = set_immigration_codes ^ set_referendum_codes
set_codes_missing

{'GI',
 'N09000001',
 'N09000002',
 'N09000003',
 'N09000004',
 'N09000005',
 'N09000006',
 'N09000007',
 'N09000008',
 'N09000009',
 'N09000010',
 'N09000011',
 'N92000002'}

In [27]:
# Let's check to which regions these codes belong
immigration_df[immigration_df["Area_Code"].isin(list(set_codes_missing))]
# Codes missing from the referendum_df but appearing in the immigration_df correspond to the areas in Northern Ireland

,Area_Code,Area,country,population_2001,births_2002,deaths_2002,natchange_2002,internal_in_2002,internal_out_2002,internal_net_2002,...,internal_out_2015,internal_net_2015,international_in_2015,international_out_2015,international_net_2015,other_change_2015,population_2015,international_in_total,international_net_total,population_change
326,N09000001,Antrim and Newtownabbey,N,128760,1669,1016,653,4356,4269,87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10956,-595,11206
327,N09000011,Ards and North Down,N,149559,1593,1409,184,4004,3325,679,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7617,226,8372
328,N09000002,"Armagh City, Banbridge and Craigavon",N,176014,2370,1395,975,3957,3300,657,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19872,10040,29697
329,N09000003,Belfast,N,328695,3857,3324,533,10469,12157,-1688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47916,909,8135
330,N09000004,Causeway Coast and Glens,N,131374,1616,1009,607,4136,3662,474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10029,-306,10929
331,N09000005,Derry City and Strabane,N,143810,2078,1033,1045,2635,2747,-112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9556,-2653,5388
332,N09000006,Fermanagh and Omagh,N,105751,1340,899,441,3220,2507,713,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10781,3564,9241
333,N09000007,Lisburn and Castlereagh,N,124585,1528,1092,436,4638,5062,-424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8100,1967,14042
334,N09000008,Mid and East Antrim,N,127452,1461,1151,310,3349,3013,336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9000,2821,9190
335,N09000009,Mid Ulster,N,119112,1742,871,871,2344,2367,-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19491,10642,23783


In [28]:
referendum_df[referendum_df["Area_Code"].isin(list(set_codes_missing))]

# On the other hand, codes missing from the immigration_df but appearing in the referendum_df belong to Gibraltar and 
# Northern Ireland as a whole

,id,Region_Code,Region,Area_Code,Area,Electorate,ExpectedBallots,VerifiedBallotPapers,Pct_Turnout,Votes_Cast,...,Remain,Leave,Rejected_Ballots,No_official_mark,Voting_for_both_answers,Writing_or_mark,Unmarked_or_void,Pct_Remain,Pct_Leave,Pct_Rejected
171,381,N92000002,Northern Ireland,N92000002,Northern Ireland,1260955,790647,790523,62.69,790523,...,440707,349442,374,18,148,1,207,55.78,44.22,0.05
308,382,E12000009,South West,GI,Gibraltar,24119,20172,20172,83.64,20172,...,19322,823,27,0,8,0,19,95.91,4.09,0.13


# Aggregating data from Northern Ireland

So, here we have a problem: while the immigration dataframe has an entry for each region in Northern Ireland, the referendum_df collated all the Northern Ireland information in a single row. To be able to work with both dataframes without losing information from Northern Ireland we will have to treat its data in a single row within our immigration_df. To do so we will aggregate data from all the Northern Ireland Regions and put it in a new row that will hold the same code as the referendum_df.
With Gibraltar there is nothing we can do. We are missing all the information regarding the referendum result.

In [29]:
# First lest's use a variable to hold only the rows affecting Northern Ireland:

northern_ireland_df = immigration_df[immigration_df["Area_Code"].isin(list(set_codes_missing))]

In [30]:
# Now let's sum the values correspondin to the numeric columns for Northern Ireland
northern_ireland_values = northern_ireland_df[northern_ireland_df.columns[3:]].sum(axis=0)
northern_ireland_values

# We see that the length of the values is smaller than the original set, since we have not included the first three columns.

population_2001            1688838
births_2002                  21460
deaths_2002                  14432
natchange_2002                7028
internal_in_2002             46693
internal_out_2002            45772
internal_net_2002              921
international_in_2002         6488
international_out_2002        6393
international_net_2002          95
other_change_2002              652
population_2002            1697534
births_2003                  21433
deaths_2003                  14648
natchange_2003                6785
internal_in_2003             49424
internal_out_2003            49322
internal_net_2003              102
international_in_2003         6810
international_out_2003        6683
international_net_2003         127
other_change_2003              376
population_2003            1704924
births_2004                  22049
deaths_2004                  14736
natchange_2004                7313
internal_in_2004             55756
internal_out_2004            55369
internal_net_2004   

In [31]:
# Here we append the missing values for the new row
new_row = pd.Series(["N92000002", "Northern Ireland", "N"]).append(northern_ireland_values)
new_row.reset_index(drop=True, inplace=True)
new_row = list(new_row)
len(new_row)

# And now we are back to the original length of the columns

161

In [32]:
# To make the work easier we use the list to create a new dataframe to append to the original one:

new_data_frame = pd.DataFrame([new_row],columns=immigration_df.columns)
new_data_frame

,Area_Code,Area,country,population_2001,births_2002,deaths_2002,natchange_2002,internal_in_2002,internal_out_2002,internal_net_2002,...,internal_out_2015,internal_net_2015,international_in_2015,international_out_2015,international_net_2015,other_change_2015,population_2015,international_in_total,international_net_total,population_change
0,N92000002,Northern Ireland,N,1688838,21460,14432,7028,46693,45772,921,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168493,33365,151660


In [33]:
# And we can now append our newly created DataFrame to the original one

immigration_df = immigration_df.append(new_data_frame, ignore_index=True)
immigration_df

# Here we have a new row that hold the aggregated values for all Northern Ireland.

,Area_Code,Area,country,population_2001,births_2002,deaths_2002,natchange_2002,internal_in_2002,internal_out_2002,internal_net_2002,...,internal_out_2015,internal_net_2015,international_in_2015,international_out_2015,international_net_2015,other_change_2015,population_2015,international_in_total,international_net_total,population_change
0,E06000001,Hartlepool,E,90152,1017,1042,-25,2140,2301,-161,...,2281,-144,248,135,113,15,92493,2524,546,2438
1,E06000002,Middlesbrough,E,141233,1663,1396,267,5109,6737,-1628,...,7071,-887,1250,450,800,12,139509,15883,8376,-2114
2,E06000003,Redcar and Cleveland,E,139159,1336,1523,-187,3951,4259,-308,...,4019,195,202,109,93,25,135275,3501,40,-4117
3,E06000004,Stockton-on-Tees,E,183795,1938,1719,219,6269,5531,738,...,6659,-217,994,557,437,-23,194803,9454,1953,10324
4,E06000005,Darlington,E,97894,1102,1173,-71,3815,3521,294,...,3779,-36,347,407,-60,19,105389,4671,304,7473
5,E06000006,Halton,E,118559,1389,1230,159,3253,3678,-425,...,3646,-176,249,217,32,16,126528,2596,-615,7795
6,E06000007,Warrington,E,191202,2149,1836,313,6517,6412,105,...,6650,477,1175,722,453,-13,207695,11838,3905,15226
7,E06000008,Blackburn with Darwen,E,138453,2126,1383,743,4792,5665,-873,...,5631,-877,778,673,105,-7,146846,10904,3822,8290
8,E06000009,Blackpool,E,142270,1417,2135,-718,9023,8333,690,...,8241,-761,565,561,4,10,139578,9475,444,-1769
9,E06000010,"Kingston upon Hull, City of",E,249913,2814,2584,230,9009,10690,-1681,...,9815,-1100,2718,1467,1251,261,258995,37535,21669,7797


In [34]:
# Let's hold the dataframe in a csv file:
# immigration_df.to_csv("data/immigration_df.csv")